In [137]:
import os
from flask import Flask
import json
import time
import auth
import pytz
import uuid
import hashlib
import socket
import numpy as np
import datetime
import sqlite3
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from pandas.io.json import json_normalize
from flask_sqlalchemy import SQLAlchemy
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


colors = [
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
    'red',
    'black',
    'blue',
    'yellow',
    'cyan',
    'pink',
    'orange',
    'green',
    'brown',
    'grey',
]

def get_datax(_db):
    conn = sqlite3.connect(_db)
    cur = conn.cursor()
    cur.execute("SELECT json_data FROM flight_data ORDER BY created DESC LIMIT 1")
    _data = cur.fetchall()
    cur.close()
    conn.close()
    return _data

def determine_direction(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS" or x == "KSC" or x == "SLD" or x == "TAT":
        return str("TAM")
    else:
        return str("SPAT")


def determine_production(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS":
        return str("BTS")
    elif x == "KSC":
        return str("KSC")
    else:
        return str("---")


def extra_catering(x):
    """

    :param x:
    :return:
    """
    try:
        return ['{0}: {1}'.format(i['code'],i['count']) for i in x]
    except:
        return 0


# def split_data_at_time(_df, _time):
#     a = _df.loc[_time:, :]
#     delimiter = a.iloc[:1].index.values[0]
#     b = _df.loc[:delimiter, :]
#     b = b.drop(delimiter)
#     return a, b, delimiter


def split_data_at_time(_df, _time):
    if 'xUID' not in _df:                                # temporarily create unique index, if does not exists
        _df['xUID'] = range(_df.shape[0])
    a = _df.loc[_time:, :]
    delimiter = a.iloc[:1].index.values[0]
    delimiter = _df[_df.index == delimiter].ix[0].xUID   # get unique index of first occurrence of delimiter
    b = _df[_df['xUID'] < delimiter]                     # split data-frame based on new index
    del _df['xUID']                                      # drop index since it's not needed anymore
    return a, b, delimiter



def get_unique_routes(_df):
    return set(_df['Route'].unique().tolist())



def get_unique_days(_data):
    """
    :param _data:
    :return:
    """
    _raw_data = json.loads((_data[0][0]).decode("utf-8"))
    _df = json_normalize(_raw_data['data']['flight']['data'])
    
    # _df = json_normalize(_data['data']['flight']['data'])
    _df_normalized = _df
    
    grouped = _df.groupby(['local_std_date'])
    _all_unique_days = []
    for day, group in grouped:
        minimum = group['local_std_time'].min()
        maximum = group['local_std_time'].max()
        _all_unique_days.append((day, minimum, maximum))

    start, end = _all_unique_days[::len(_all_unique_days) - 1]
    start = list(start[:2])
    end = list(end[::len(end) - 1])
    start = datetime.datetime.strptime(start[0] + ' ' + start[1], '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end[0] + ' ' + end[1], '%Y-%m-%d %H:%M:%S')
    final = []
    a = start
    while True:
        b = (a + datetime.timedelta(1)).replace(hour=9, minute=0)
        if b > end:
            b = end
            final.append('{}___{}'.format(str(a), str(b)))
            break
        final.append('{}___{}'.format(str(a), str(b)))
        a = b
    return final, _df



 
def hash_column(_string):
    # uuid is used to generate a random number
    salt = uuid.uuid4().hex
    return hashlib.sha256(salt.encode() + _string.encode()).hexdigest() + ':' + salt

def undo_hash(hashed_password, user_password):
    password, salt = hashed_password.split(':')
    return password == hashlib.sha256(salt.encode() + user_password.encode()).hexdigest()


def render_tables(_df_normalized):
    """

    :param _df_normalized:
    :return:
    """
    _df = _df_normalized
    _compare = pd.DataFrame()
    unique = _df['route_id'].unique().tolist()
    color_map = {str(u): c for u, c in zip(unique, colors)}

    _compare['Departure'] = pd.to_datetime(_df['local_std_date'] + ' ' + _df['local_std_time'])
    _compare['Depart'] = _compare['Departure']
    _compare[''] = _df['route_id'].map(
        lambda x: '<span style="border-left: 12px solid {0};"></span>'.format(color_map[str(x)]))
    _compare['Arrival'] = pd.to_datetime(_df['local_sta_date'] + ' ' + _df['local_sta_time'])
    _compare['Flight'] = _df['flight_number']
    _compare['Aircraft'] = _df['aircraft_config']
    _compare['Hash'] = _df['aircraft_reg'].astype(str)  + _df['route_id'].astype(str)  + _compare['Departure'].astype(str) 
    _compare['Hash'] = _compare['Hash'].map(hash_column)
    _compare['Reg'] = _df['aircraft_reg']
    _compare['Route'] = _df['route_id']
    _compare["Meal"] = _df["catering_order.flight_meal_type"]
    _compare['Direction'] = _df['departure_iata'].map(determine_direction)
    _compare['Production'] = _df['departure_iata'].map(determine_production)
    _compare['From'] = _df['departure_iata']
    _compare['To'] = _df['destination_iata']

    _compare['Quantity'] = _df['catering_order.quantity_y']
    _compare['Crew'] = _df['catering_order.quantity_crew']
    _compare['Extra Catering'] = _df['extra_catering'].map(extra_catering)
    _compare['Note'] = _df['catering_order.general_note'].map(
        lambda x: "{0}{1}{2}".format("""<div class="hoverable">""", str(x), "</div>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\n', "<br>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\r', ''))
    _compare['Quantity'] = _compare['Quantity'].fillna(0)
    _compare['Crew'] = _compare['Crew'].fillna(0)
    _compare.index = _compare['Departure']
    return _compare




def select_scoped_timeframe(_compare, _all_unique_days, _day):
    _day_dict_lookup = {i.replace(' ', '_').replace(':', '_'): i for i in _all_unique_days}
    DAY = _day_dict_lookup[_day]
    FT, ST = DAY.split('___')
    valid, invalid, dm1 = split_data_at_time(_compare, FT)
    extra, main, dm2 = split_data_at_time(valid, ST)
    valid_rids = get_unique_routes(valid)
    invalid_rids = get_unique_routes(invalid)
    intersection = invalid_rids & valid_rids
    main_rids = get_unique_routes(main)
    main_rids = main_rids - intersection
    final = valid[valid['Route'].isin(list(main_rids))]
    return final





In [138]:
db = "2w.sqlite"
data = get_datax(db)
ud, normal = get_unique_days(data)
compare = render_tables(normal)
compare = compare.sort_values(['Depart'], ascending=[True])

In [139]:
ud

['2018-07-25 02:40:00___2018-07-26 09:00:00',
 '2018-07-26 09:00:00___2018-07-27 09:00:00',
 '2018-07-27 09:00:00___2018-07-28 09:00:00',
 '2018-07-28 09:00:00___2018-07-29 00:35:00']

In [140]:
x = {i.replace(' ', '_').replace(':', '_'): i for i in ud}

In [141]:
compare.shape[0]

164

In [142]:
sliced = []
for i in x.keys():
    sliced.append(select_scoped_timeframe(compare, ud, i))
    

C:\tools\miniconda3\lib\site-packages\ipykernel_launcher.py:225: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [147]:
hashes = []
for i in sliced:
    hashes.append(set(i['Hash'].unique().tolist())) 
    
    

In [152]:
hashes[0] & hashes[1]
hashes[1] & hashes[2]
hashes[2] & hashes[3]


set()

In [143]:
pd.concat(sliced).shape

(164, 18)

In [156]:
 
def hash_column(_string):
    # uuid is used to generate a random number
    salt = uuid.uuid4().hex
    return hashlib.sha256(salt.encode() + _string.encode()).hexdigest() + ':' + salt

def undo_hash(hashed_password, user_password):
    password, salt = hashed_password.split(':')
    return password == hashlib.sha256(salt.encode() + user_password.encode()).hexdigest()

In [160]:
a = hash_column('jahoda')
b = hash_column('jablko')

check_password(a, b)


False